Line/Space Function

In [8]:
# Import the needed extensions
import math
import klayout
import pya

def ls_cell(tone="D",size=100,pitch=2000,dimension=10000,angle=45,x2y=1):

    #Create the layout
    layout = pya.Layout()

    #Name the topcell
    Unit_main = layout.create_cell(f"LS_Array_{tone}_{size}_{pitch}_{angle}")

    #Create sub-cells
    Unit_1 = layout.create_cell(f"{tone}_{size}nm_line")
    Unit_2 = layout.create_cell(f"{pitch}nm_pitch")

    #Defines initial layers
    l_outline = layout.layer(11,0) #overlay layer for the cell
    l_line = layout.layer(2,0) #line layer, sacrificial

    #Create the overlay shape
    overlay = Unit_main.shapes(l_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))

    #Check pitch of the LS array
    pitch_check = math.floor(dimension/pitch)
    iso = pitch_check < 2 #boolean
    bar3 = pitch-size < size/2

    #Define line dimensions
    l_left = -size/2
    l_bottom = -dimension/2
    l_right = size/2
    l_top = dimension/2
    l_2bottom = -dimension
    l_2top = dimension

    #Create line
    line_vert = Unit_1.shapes(l_line).insert(pya.Box(l_left, l_bottom, l_right, l_top))
    
    #Define boundary units
    if iso:
        line_bound = Unit_2.shapes(l_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))
        a1 = pya.Region(Unit_1.shapes(l_line))
        a2 = pya.Region(Unit_2.shapes(l_outline))
        axor = a1 ^ a2

    #Create the LS array
    if iso:
        line_vert = Unit_1.shapes(l_line).insert(pya.Box(l_left, l_bottom, l_right, l_top))
        line_bound = Unit_2.shapes(l_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))
    elif bar3:
        line_vert = Unit_1.shapes(l_line).insert(pya.Box(l_left, l_bottom, l_right, l_top))
        line_bound = Unit_2.shapes(l_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))
    else:
        for num in range(-pitch_check,pitch_check,1):
                coord_x = num*pitch
                line_vert = Unit_main.shapes(l_line).insert(pya.Box(coord_x+l_left,l_2bottom,coord_x+l_right,l_2top))


    #Does the angle transformation for the lines for rotations
    t = pya.ICplxTrans(1,angle,0,0,0)
    Unit_main.shapes(l_line).transform(t)

    #Removes polygons extending beyond the overlay layer by only including those within the overlay region (& statement)
    r1 = pya.Region(Unit_main.shapes(l_line))
    r2 = pya.Region(Unit_main.shapes(l_outline))
    r_and = r1 & r2

    #Flips tone using XOR if tone is C instead of D
    if tone == "C":
        r_and = r_and ^ r2

    #Adds new layer with finished cell
    l_diff = layout.layer(1,0)
    r_diff = Unit_main.shapes(l_diff).insert(r_and)

    #Removes sacrificial layer
    l_remove = layout.delete_layer(1)

    #Export GDS
    layout.write(f"LS_test_t{tone}_s{size}_p{pitch}_l{dimension}_a{angle}.gds")

ls_cell()

Contact Function

In [ ]:
# Import the needed extensions
import math
import klayout
import pya

def contact_cell(tone="D",size=100,pitch=200,dimension=10000,angle=0,x2y=3):

    #Creates the layout and names it
    layout = pya.Layout()
    Unit = layout.create_cell(f"HD_Array_{tone}_{size}_{pitch}_{angle}_{x2y}to1")

    #Defines initial layers
    l_outline = layout.layer(11,0) #overlay layer for the cell
    l_cont = layout.layer(2,0) #contact layer, sacrificial

    #Create the overlay shape
    overlay = Unit.shapes(l_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))

    #Define the true pitch, after taking x2y into account
    true_pitch_x = (pitch-size) + (size*x2y)
    true_pitch_y = (pitch)

    #Check pitch of the cont array
    pitch_check = math.floor(dimension/true_pitch_y)
    iso = pitch_check < 2 #boolean

    #Set contact dimensions
    c_left = -size*x2y/2
    c_bottom = -size/2
    c_right = size*x2y/2
    c_top = size/2

    #Create the cont array
    if iso:
        cont = Unit.shapes(l_cont).insert(pya.Box(c_left,c_bottom,c_right,c_top))
    else:
        for i in range(-pitch_check,pitch_check,1):
            for j in range(-pitch_check,pitch_check,1):
                coord_x = i*true_pitch_x
                coord_y = j*true_pitch_y
                cont = Unit.shapes(l_cont).insert(pya.Box(coord_x+c_left,coord_y+c_bottom,coord_x+c_right,coord_y+c_top))


    #Does the angle transformation for the holes/dots for rotations
    t = pya.ICplxTrans(1,angle,0,0,0)
    Unit.shapes(l_cont).transform(t)

    #Removes polygons extending beyond the overlay layer by only including those within the overlay region (& statement)
    r1 = pya.Region(Unit.shapes(l_cont))
    r2 = pya.Region(Unit.shapes(l_outline))
    r_and = r1 & r2
    
    #Flips tone using XOR if tone is C instead of D
    if tone == "C":
        r_and = r_and ^ r2
    
    #Adds new layer with finished cell
    l_diff = layout.layer(1,0)
    r_diff = Unit.shapes(l_diff).insert(r_and)

    #Removes sacrificial layer
    l_remove = layout.delete_layer(1)

    #Export GDS
    layout.write(f"Contact_test_t{tone}_s{size}_p{pitch}_l{dimension}_a{angle}_{x2y}to1.gds")

contact_cell()

Line with SRAF function

In [1]:
# Import the needed extensions
import math
import klayout
import pya

def lsraf_cell(tone="C",size=300,pitch=100,dimension=5000,angle=45,sraf=60,sraf_num=2):

    #Creates the layout and names it
    layout = pya.Layout()
    Unit = layout.create_cell(f"LSRAF_Array_{tone}_{size}_{pitch}_{angle}_{sraf}_{sraf_num}")

    #Defines the initial layers
    lsraf_outline = layout.layer(11,0) #overlay layer for the cell
    lsraf_line = layout.layer(2,0) #line layer, sacrificial

    #Create the overlay shape
    overlay = Unit.shapes(lsraf_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))

    #Define vertical line dimensions and both sets of SRAF dimensions
    l_left = -size/2
    l_bottom = -(dimension-(sraf_num+1)*(pitch+sraf))/2
    l_right = size/2
    l_top = (dimension-(sraf_num+1)*(pitch+sraf))/2
    sraf_side_left = -sraf/2
    sraf_side_bottom = l_bottom
    sraf_side_right = sraf/2
    sraf_side_top = l_top
    sraf_end_left = l_left
    sraf_end_bottom = -sraf/2
    sraf_end_right = l_right
    sraf_end_top = sraf/2

    #Create the structure
    line_vert = Unit.shapes(lsraf_line).insert(pya.Box(l_left, l_bottom, l_right, l_top))
    for num in range(-sraf_num,sraf_num+1,1):
        coord = num*pitch
        if coord < 0:
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(l_left+(coord+sraf_side_left),sraf_side_bottom,l_left+(coord+sraf_side_right),sraf_side_top))
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(sraf_end_left,l_bottom+(coord+sraf_end_bottom),sraf_end_right,l_bottom+(coord+sraf_end_top)))
        elif coord > 0:
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(l_right+(coord+sraf_side_left),sraf_side_bottom,l_right+(coord+sraf_side_right),sraf_side_top))
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(sraf_end_left,l_top+(coord+sraf_end_bottom),sraf_end_right,l_top+(coord+sraf_end_top)))

    #Does the angle transformation for the lines for rotations
    t = pya.ICplxTrans(1,angle,0,0,0)
    Unit.shapes(lsraf_line).transform(t)

    #Removes polygons extending beyond the overlay layer by only including those within the overlay region (& statement)
    r1 = pya.Region(Unit.shapes(lsraf_line))
    r2 = pya.Region(Unit.shapes(lsraf_outline))
    r_and = r1 & r2
    
    #Flips tone using XOR if tone is C instead of D
    if tone == "C":
        r_and = r_and ^ r2
    
    #Adds new layer with finished cell
    l_diff = layout.layer(1,0)
    r_diff = Unit.shapes(l_diff).insert(r_and)

    #Removes sacrificial layer
    l_remove = layout.delete_layer(1)

    #Export GDS
    layout.write(f"LSRAF_test_t{tone}_s{size}_p{pitch}_l{dimension}_a{angle}_sraf{sraf}_numsrafs{sraf_num}.gds")

lsraf_cell()

Curvilinear Function

In [11]:
# Import the needed extensions
import math
import klayout
import pya

#Going to make this a new PCell for Klayout to use, I essentially copied this from here: (https://www.youtube.com/watch?v=2e6DuFj0Xws)
class Spiral(pya.PCellDeclarationHelper):

    def __init__(self):
    
        #Initiallizes the super class
        super(Spiral, self).__init__()

        #Declare the parameters
        self.param("l", self.TypeLayer, "Layer", default = pya.LayerInfo(1,0)) #sets up the layer
        self.param("n", self.TypeInt, "Number of points", default = 64)
        self.param("inner_r", self.TypeDouble, "Inner Radius", default = 1)
        self.param("outer_r", self.TypeDouble, "Outer Radius", default = 10)
        self.param("width", self.TypeDouble, "Width", default = 1)
        self.param("spacing", self.TypeDouble, "Spacing", default = 1)
        self.param("inner_handle", self.TypeShape, "", default = pya.DPoint(0,0)) #handle used to change radius independent of radius parameter
        self.param("outer_handle", self.TypeShape, "", default = pya.DPoint(0,0)) 

        #Additional hidden parameters for whether the radius has changed or the "s" handle has been moved
        self.param("inner_memory", self.TypeDouble, "Inner Memory", default = 0.0, hidden = True)
        self.param("outer_memory", self.TypeDouble, "Outer Memory", default = 0.0, hidden = True)

    def display_text_impl(self):
        #Provides a descriptive text for the cell
        return "Spiral(L = " + str(self.l) + ", Outer radius = " + ('%.3f' % self.outer_r) + ")"
    
    def coerce_parameters_impl(self):

        #We employ coerce_parameters_impl to decide wheter the handle or the numeric paramter has changed
        #(by comparing against the effective radius ru)
        #and set ru to the effective radius. We also update the numerical value or the shape,
        #depending on which one has not changed

        inner_handle_radius = None
        outer_handle_radius = None

        if isinstance(self.inner_handle, pya.DPoint):
            #if inner radius manually changed on layout by user, calculates value, computes distance in microns from 0,0 point
            inner_handle_radius = self.inner_handle.distance(pya.DPoint(0,0))
        if isinstance(self.outer_handle, pya.DPoint):
            #if inner radius manually changed on layout by user, calculates value, computes distance in microns from 0,0 point
            inner_handle_radius = self.outer_handle.distance(pya.DPoint(0,0))
        
        if abs(self.inner_r - self.inner_memory) < 1e-6 and abs(self.outer_r - self.outer_memory) < 1e-6: #case where either handle changes
            self.inner_memory = inner_handle_radius
            self.inner_r = inner_handle_radius
            self.outer_memory = outer_handle_radius
            self.outer_r = outer_handle_radius
        else: #case where the parameter changes
            self.inner_memory = self.inner_r
            self.inner_handle = pya.DPoint(-self.inner_r,0)
            self.outer_memory = self.outer_r
            self.outer_handle = pya.DPoint(-self.outer_r,0)
        
        #n must be larger than or equal to 4
        if self.n <= 4:
            self.n = 4
    
    def can_create_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: we can use any shape which has a finite bounding box
        return self.shape.is_box() or self.shape.is_polygon() or self.shape.is_path()
    
    def parameters_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: we set the inner and outer radii based on the bounding box of the shape being used
        self.inner_r = self.shape.bbox().width() * self.layout.dbu / 4
        self.outer_r = self.shape.bbox().width() * self.layout.dbu / 2
        self.l = self.layout.get_info(self.layer) #puts the new spiral shape on the same layer as the shape being converted
    
    def transformation_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: places the spiral in the same position as the converted shape
        return pya.Trans(self.shape.bbox().center())
    
    def produce_impl(self):
        #This is the main part of the function, where the shape is actually formed/created

        #Fetch the parameters, and convert to the database units being used in the layout
        inner_r_dbu = self.inner_r / self.layout.dbu
        outer_r_dbu = self.outer_r / self.layout.dbu

        #Set up array for the points to be placed for the spiral
        pts = []
        #The change in angle (in radians) being stepped across to drive the spiral
        da = math.pi * 2 / self.n
        #The change in radius being stepped across to expand the spiral.
        #After one rotation, want the spiral to have expanded by one width and one spacing.
        dr = (self.width+self.spacing)/self.n/self.layout.dbu

        #Initial conditions for spiral loop.
        current_radius = inner_r_dbu
        current_angle = 0

        #While loop used to fill out the spiral
        while current_radius < outer_r_dbu:
            #Adds a new point to the pts array for each while loop pass
            #Cosine and sine calculations converting the angle in radians to x,y coordinates
            pts.append(pya.Point.from_dpoint(pya.Dpoint(current_radius * math.cos(current_angle), current_radius * math.sin(current_angle))))
            #Steps the radius up
            current_radius += dr
            #Steps up the current angle.
            #Modulus of 2pi is being used to avoid angle buildup after 2pi is reached, reverting back to 0 instead
            current_angle = (current_angle+da)%(math.pi*2)
        
        #Creates the shape
        self.cell.shapes(self.l_layer).insert(pya.Path(pts, self.width/self.layout.dbu))

class MyLib(pya.Library):
    #The library where we put the PCell into
    def __init__(self):
        
        #Set the description
        self.description = "My Library"

        #Create the PCell declarations
        self.layout().register_pcell("Spiral", Spiral())

        #Register with the nam "MyLib"
        self.register("MyLib")

#Instantiate and register the library
MyLib()